## Import Libraries

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import time

print(f"PyTorch version: {torch.__version__}")

## 1. LU Decomposition

**LU Decomposition:** A = LU where L is lower triangular and U is upper triangular

In [ ]:
A = torch.tensor([[4., 3., 2.], [3., 2., 1.], [2., 1., 3.]])
print(f"Original matrix A:\n{A}")

P, L, U = torch.linalg.lu(A)
print(f"\nPermutation P:\n{P}")
print(f"\nLower L:\n{L}")
print(f"\nUpper U:\n{U}")
print(f"\nReconstruction P@L@U:\n{P @ L @ U}")
print(f"\nReconstruction error: {torch.linalg.norm(A - P @ L @ U):.6e}")

## 2. QR Decomposition

**QR Decomposition:** A = QR where Q is orthogonal (Q'Q = I) and R is upper triangular

In [ ]:
A = torch.randn(5, 3)
print(f"Original matrix A (5x3):\n{A}")

Q, R = torch.linalg.qr(A)
print(f"\nOrthogonal Q (5x3):\n{Q}")
print(f"\nUpper triangular R (3x3):\n{R}")
print(f"\nOrthogonality check Q'Q:\n{Q.T @ Q}")
print(f"\nReconstruction error: {torch.linalg.norm(A - Q @ R):.6e}")

## 3. Singular Value Decomposition (SVD)

**SVD:** A = UΣV' where U and V are orthogonal, Σ is diagonal with singular values

In [ ]:
A = torch.randn(4, 6)
print(f"Original matrix A (4x6)")

U, S, Vh = torch.linalg.svd(A, full_matrices=False)
print(f"\nU shape: {U.shape}")
print(f"S shape: {S.shape}")
print(f"Vh shape: {Vh.shape}")
print(f"\nSingular values: {S}")

# Reconstruct
A_reconstructed = U @ torch.diag(S) @ Vh
print(f"\nReconstruction error: {torch.linalg.norm(A - A_reconstructed):.6e}")

## 4. Eigenvalue Decomposition

**Eigenvalue Decomposition:** A = VΛV⁻¹ where Λ is diagonal with eigenvalues, V contains eigenvectors

In [ ]:
A = torch.tensor([[4., 1.], [2., 3.]])
print(f"Matrix A:\n{A}")

eigenvalues, eigenvectors = torch.linalg.eig(A)
print(f"\nEigenvalues: {eigenvalues}")
print(f"\nEigenvectors:\n{eigenvectors}")

# Verify: A*v = λ*v
v1 = eigenvectors[:, 0]
lambda1 = eigenvalues[0]
print(f"\nVerification A*v1: {A @ v1}")
print(f"λ1*v1: {lambda1 * v1}")

## 5. Cholesky Decomposition

**Cholesky Decomposition:** A = LL' for positive definite matrices, L is lower triangular

In [ ]:
# Create positive definite matrix
A = torch.randn(4, 4)
A_pd = A @ A.T + torch.eye(4) * 0.1
print(f"Positive definite matrix:\n{A_pd}")

L = torch.linalg.cholesky(A_pd)
print(f"\nLower triangular L:\n{L}")
print(f"\nReconstruction L@L':\n{L @ L.T}")
print(f"\nReconstruction error: {torch.linalg.norm(A_pd - L @ L.T):.6e}")

## 6. Low-Rank Approximation with SVD

Approximate a matrix using only its top-k singular values

In [ ]:
A = torch.randn(10, 10)
print(f"Original matrix rank: {torch.linalg.matrix_rank(A)}")

U, S, Vh = torch.linalg.svd(A, full_matrices=False)

# Test different ranks
for rank in [1, 3, 5]:
    U_k = U[:, :rank]
    S_k = S[:rank]
    Vh_k = Vh[:rank, :]
    
    A_approx = U_k @ torch.diag(S_k) @ Vh_k
    error = torch.linalg.norm(A - A_approx, ord='fro')
    rel_error = error / torch.linalg.norm(A, ord='fro')
    
    print(f"\nRank-{rank} approximation:")
    print(f"  Frobenius error: {error:.4f}")
    print(f"  Relative error: {rel_error:.4f} ({rel_error*100:.2f}%)")

## 7. Image Compression with SVD

In [ ]:
# Create synthetic image
x = torch.linspace(-5, 5, 200)
y = torch.linspace(-5, 5, 200)
X, Y = torch.meshgrid(x, y, indexing='ij')
image = torch.sin(X) * torch.cos(Y) + 0.5 * torch.randn(200, 200)

print(f"Image shape: {image.shape}")

# Compute SVD
U, S, Vh = torch.linalg.svd(image, full_matrices=False)

# Test different compression ranks
ranks = [5, 10, 20, 50, 100]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

# Original image
axes[0].imshow(image.numpy(), cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')

# Compressed images
for idx, rank in enumerate(ranks, start=1):
    U_k = U[:, :rank]
    S_k = S[:rank]
    Vh_k = Vh[:rank, :]
    A_approx = U_k @ torch.diag(S_k) @ Vh_k
    
    error = torch.linalg.norm(image - A_approx, ord='fro')
    rel_error = error / torch.linalg.norm(image, ord='fro')
    
    axes[idx].imshow(A_approx.numpy(), cmap='gray')
    axes[idx].set_title(f'Rank {rank}\nError: {rel_error:.4f}')
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

## 8. Singular Value Decay

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.semilogy(S.numpy(), 'o-', linewidth=2)
ax.set_xlabel('Index', fontsize=12)
ax.set_ylabel('Singular Value', fontsize=12)
ax.set_title('Singular Value Decay', fontsize=14)
ax.grid(True, alpha=0.3)
plt.show()

## 9. Solving Linear Systems

Solve Ax = b using various methods

In [ ]:
A = torch.randn(3, 3)
b = torch.randn(3)

# Direct solve
x = torch.linalg.solve(A, b)
print(f"Solution x: {x}")
print(f"\nVerification A @ x: {A @ x}")
print(f"b: {b}")
print(f"\nSolution error: {torch.linalg.norm(A @ x - b):.6e}")

## 10. Pseudo-Inverse (Moore-Penrose)

For rectangular or singular matrices

In [ ]:
A = torch.randn(3, 5)
A_pinv = torch.linalg.pinv(A)

print(f"A shape: {A.shape}")
print(f"A_pinv shape: {A_pinv.shape}")
print(f"\nA @ A_pinv @ A ≈ A:")
print(f"Reconstruction error: {torch.linalg.norm(A @ A_pinv @ A - A):.6e}")

## 11. Matrix Rank

In [ ]:
# Full rank matrix
A_full = torch.randn(5, 5)
print(f"Full rank matrix (5x5): rank = {torch.linalg.matrix_rank(A_full)}")

# Rank deficient matrix
A_rank_def = torch.tensor([[1., 2., 3.], [2., 4., 6.], [3., 6., 9.]])
print(f"\nRank deficient matrix:\n{A_rank_def}")
print(f"Rank: {torch.linalg.matrix_rank(A_rank_def)}")

## 12. Benchmark Different Decompositions

In [ ]:
sizes = [10, 50, 100, 200]
results = {'LU': [], 'QR': [], 'SVD': [], 'Eigen': [], 'Cholesky': []}

for n in sizes:
    print(f"\nMatrix size: {n}x{n}")
    
    # LU
    A = torch.randn(n, n)
    start = time.time()
    _ = torch.linalg.lu(A)
    results['LU'].append(time.time() - start)
    print(f"  LU: {results['LU'][-1]:.4f}s")
    
    # QR
    start = time.time()
    _ = torch.linalg.qr(A)
    results['QR'].append(time.time() - start)
    print(f"  QR: {results['QR'][-1]:.4f}s")
    
    # SVD
    start = time.time()
    _ = torch.linalg.svd(A, full_matrices=False)
    results['SVD'].append(time.time() - start)
    print(f"  SVD: {results['SVD'][-1]:.4f}s")
    
    # Eigen
    start = time.time()
    _ = torch.linalg.eig(A)
    results['Eigen'].append(time.time() - start)
    print(f"  Eigen: {results['Eigen'][-1]:.4f}s")
    
    # Cholesky (positive definite)
    A_pd = A @ A.T + torch.eye(n) * 0.1
    start = time.time()
    _ = torch.linalg.cholesky(A_pd)
    results['Cholesky'].append(time.time() - start)
    print(f"  Cholesky: {results['Cholesky'][-1]:.4f}s")

## 13. Plot Benchmark Results

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
for method, times in results.items():
    ax.plot(sizes, times, 'o-', label=method, linewidth=2, markersize=8)

ax.set_xlabel('Matrix Size', fontsize=12)
ax.set_ylabel('Time (seconds)', fontsize=12)
ax.set_title('Decomposition Method Benchmarks', fontsize=14)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_yscale('log')
plt.show()